<a href="https://colab.research.google.com/github/jammy-bot/dask-v-pandas/blob/master/dask-v-pandas-test_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dask Acceleration vs Pandas

This notebook explores speed advantages realized by using the Dask library over Pandas for dataframe operations in Python.

Local system:

| Tech | Version |
| --- | --- |
| Python | 3.6.9 |
| jupyter-notebook | 6.0.1 |
| pandas | 1.0.3 |
| dask | 2.17.2 |
| x64 - based PC | i7 - 2.60 GHz |
| RAM | 16.0 GB |

Hat - tip to Saturn Cloud's [Your Practical Guide to Dask](https://www.saturncloud.io/s/practical-guide-to-dask/).

In [1]:
# importing library to measure execution time
import timeit
from timeit import default_timer as timer

# starting timer for complete notebook execution
notebook_start = timer()

In [2]:
# importing required libraries
import random
import pandas as pd
import os # for directory operations

__Randomize stock data in Python and save the data as a Pandas dataframe.__

In [3]:
# instantiating 1M random stock records
num_rows = 1000000

symbols = ["AAPL", "AMD", "GOOG", "MSFT", "NVDA"]
prices = [random.randint(1, 500) for _ in range(50)]

In [4]:
def get_stock_data(symbols, prices):
    '''
    function to generate random stock data from the
    `symbols` list and the randomized `prices` list
    '''
    return {"symbol": random.sample(symbols, 1)[0],
            "price": random.sample(prices, 1)[0]}

In [5]:
# using the function to generate stock data for the
# number of rows instantiated in `num_rows`
stock_data = [get_stock_data(symbols, prices) for _ in range(num_rows)]

# instantiate data as a pandas dataframe
stock_df = pd.DataFrame(stock_data,
                        columns=["symbol", "price"])

__Export stock data to a csv file.__

In [6]:
# save `stock_df` as a csv file in the data subdirectory
if not os.path.exists('data'):
    os.mkdir('data')

# prefix filename with '__rc__' to .gitignore
stock_df.to_csv("data/_rc_stock_data.csv")

# preview the dataframe
stock_df.head()

,symbol,price
0,NVDA,211
1,AMD,362
2,AAPL,191
3,AMD,443
4,NVDA,350


## Load csv Data to a Dask Dataframe

In [7]:
import dask.dataframe as dd

# loading csv data to a dask dataframe
dask_df = dd.read_csv("data/_rc_stock_data.csv")

__Repartition dask data to sizes of 100MB-or-less, per [official documentation](https://docs.dask.org/en/latest/dataframe-best-practices.html#repartition-to-reduce-overhead)__.

In [8]:
# repartitioning dask dataframe from csv
dask_df = dask_df.repartition(partition_size="100MB")

# loading pandas dataframe from csv
pandas_df = pd.read_csv("data/_rc_stock_data.csv")

## Compute CPU Acceleration with Dask vs Pandas

### Calculate Mean

__Create functions to calculate mean for the same dataframe in each of the two libraries.__

In [9]:
def pandas_mean():
    '''Calculates mean using pandas
    ''' 
    pandas_df["price"].mean()
    
def dask_mean():
    '''Calculate means using dask
    ''' 
    dask_df["price"].mean()

__Compare Calculating Means (single iteration)__

In [10]:
print("\npandas execution time: ", round(
    timeit.timeit(pandas_mean, number=1), 5)
     )
print("-"*30)

print("\ndask execution time: ", round(
    timeit.timeit(dask_mean, number=1), 5)
     )


pandas execution time:  0.00621
------------------------------

dask execution time:  0.00205


In [11]:
# computing and comparing task execution with `timer()`
start = timer()

# Calculate mean using pandas
pandas_df["price"].mean()

end = timer()

# print time elapsed in seconds
print("pandas execution time: ", round(end - start, 5))
print("-"*30)

start = timer()

# Calculate mean using Dask
dask_df["price"].mean()

end = timer()

# print time elapsed in seconds
print("\ndask execution time: ", round(end - start, 5))

pandas execution time:  0.00191
------------------------------

dask execution time:  0.00304


__Compare Calculating Means (10K iterations)__

In [12]:
print("\npandas execution time: ", round(
    timeit.timeit(pandas_mean, number=10000), 2)
     )
print("-"*30)

print("\ndask execution time: ", round(
    timeit.timeit(dask_mean, number=10000), 2)
     )


pandas execution time:  15.19
------------------------------

dask execution time:  12.71


__Compare Calculating Means (100K iterations)__

In [13]:
# computing means
p_mean = round(timeit.timeit(pandas_mean, number=100000), 2)
d_mean = round(timeit.timeit(dask_mean, number=100000), 2)

In [14]:
# comparing means
print("\npandas execution time: ", p_mean)
print("-"*30)
print("\ndask execution time: ", d_mean)

print("\nCalculating means is about",
      round((p_mean - d_mean) / p_mean*100, 2),
      "% faster over a large dataset, with dask on CPU.")


pandas execution time:  149.5
------------------------------

dask execution time:  127.39

Calculating means is about 14.79 % faster over a large dataset, with dask on CPU.


### Filter Dataframe

__Create functions to filter dataframe in each of the two libraries.__

In [15]:
def pandas_filter():
    '''Filters by price using pandas
    ''' 
    pandas_df[pandas_df["price"] > 250]
    
def dask_filter():
    '''Filters by price using dask
    ''' 
    dask_df[dask_df["price"] > 250]

__Compare Filtering Dataframes (single iteration)__

In [16]:
print("\npandas execution time: ", round(
    timeit.timeit(pandas_filter, number=1), 5)
     )
print("-"*30)

print("\ndask execution time: ", round(
    timeit.timeit(dask_filter, number=1), 5)
     )


pandas execution time:  0.02717
------------------------------

dask execution time:  0.00147


In [17]:
# computing and comparing task execution with `timer()`
start_p = timer()
# Filtering by price in Pandas
pandas_df[pandas_df["price"] > 250]
end_p = timer()
pandas_time = round(end_p - start_p, 5)

print("pandas execution time: ", pandas_time)
print("-"*30)

start_d = timer()
# Filtering by price in Dask
dask_df[dask_df["price"] > 250]
end_d = timer()
dask_time = round(end_d - start_d, 5)

print("\ndask execution time: ", dask_time)

pandas execution time:  0.0221
------------------------------

dask execution time:  0.00184


__Compare Filtering Dataframes (10K iterations)__

In [18]:
print("\npandas execution time: ", round(
    timeit.timeit(pandas_filter, number=10000), 2)
     )
print("-"*30)

print("\ndask execution time: ", round(
    timeit.timeit(dask_filter, number=10000), 2)
     )


pandas execution time:  194.62
------------------------------

dask execution time:  8.62


__Compare Filtering Dataframes (100K iterations)__

In [19]:
# computing filter time
p_filter = round(timeit.timeit(pandas_filter, number=100000), 2)

print("\npandas execution time: ", p_filter)
print("-"*30)

d_filter = round(timeit.timeit(dask_filter, number=100000), 2)

print("\ndask execution time: ", d_filter)


pandas execution time:  1938.86
------------------------------

dask execution time:  87.08


In [20]:
# comparing filter execution time
print("\nFiltering is about",
      round((p_filter - d_filter) / p_filter*100, 2),
      "% faster over a large dataset, with dask on CPU.")


Filtering is about 95.51 % faster over a large dataset, with dask on CPU.


### Add Dataframes

__Create functions to add dataframes in each of the two libraries.__

In [21]:
def pandas_add():
    '''Adds dataframes together using pandas
    ''' 
    pandas_df + pandas_df + pandas_df + pandas_df + pandas_df
    
def dask_add():
    '''Adds dataframes together using dask
    ''' 
    dask_df + dask_df + dask_df + dask_df + dask_df

__Compare Adding Dataframes (single iteration)__

In [22]:
print("\npandas execution time: ", round(
    timeit.timeit(pandas_add, number=1), 5)
     )
print("-"*30)

print("\ndask execution time: ", round(
    timeit.timeit(dask_add, number=1), 5)
     )


pandas execution time:  0.65694
------------------------------

dask execution time:  0.02987


In [23]:
start_p = timer()
# adding big dataframes in pandas
pandas_df + pandas_df + pandas_df + pandas_df + pandas_df
end_p = timer()
pandas_time = round(end_p - start_p, 5)
print("pandas execution time: ", pandas_time)
print("-"*30)

start_d = timer()
# adding big dataframes in dask
dask_df + dask_df + dask_df + dask_df + dask_df
end_d = timer()
dask_time = round(end_d - start_d, 5)
print("\ndask execution time: ", dask_time)

# calculating dask speed improvement
print("-"*30, "\n")

print(round((pandas_time - dask_time) / pandas_time*100, 3), 
      "% increase in speed")

pandas execution time:  0.51555
------------------------------

dask execution time:  0.02756
------------------------------ 

94.654 % increase in speed


__Compare Adding Dataframes (10K iterations)__

In [24]:
# compute
p_add = timeit.timeit(pandas_add, number=10000)

print("\npandas execution time: ", round(
    p_add, 2)
     )
print("-"*30)

d_add = timeit.timeit(dask_add, number=10000)

print("\ndask execution time: ", round(
    d_add, 2)
     )

print("\nAdding dataframes is about",
      round((p_add - d_add) / p_add*100, 2),
      "% faster over a large dataset, with dask on CPU.")


pandas execution time:  5076.81
------------------------------

dask execution time:  243.15

Adding dataframes is about 95.21 % faster over a large dataset, with dask on CPU.


In [25]:
# ending timer for complete notebook execution
notebook_end = timer()

# calculating time for complete notebook execution
notebook_duration = notebook_end - notebook_start
print(round(notebook_duration, 2))

7865.28


__Compare Adding Dataframes (100K iterations)__

In [26]:
# compute
# --time prohibitive--
# p_add = timeit.timeit(pandas_add, number=100000)

# compute for dask, only, in seconds
d_add = timeit.timeit(dask_add, number=100000)
d_add

2463.9490558359994

In [27]:
# print("\npandas execution time: ", round(
#     p_add, 2)
#      )
# print("-"*30)

print("\ndask execution time: ", round(
    d_add), 2)

print("\nAdding dataframes takes about",
      round(d_add/60, 2),
      "minutes over the large datasets, with dask on CPU.")


dask execution time:  2464 2

Adding dataframes takes about 41.07 minutes over the large datasets, with dask on CPU.


======================================================

## Compute GPU Acceleration with Dask vs Pandas

__The cuDF library enables GPU acceleration for Pandas - Dask dataframe computation.__

In [28]:
# import cudf

# # reinstantiate `dask_df` for use with GPU backend
# dask_df = dask_df.map_partitions(cudf.from_pandas) 

### ======_cuDF no longer available via pip [or for Windows???]_ ======

__Resume in containerized environment.__

In [29]:
# ending timer for complete notebook execution
notebook_end = timer()

# calculating time for complete notebook execution
notebook_duration = notebook_end - notebook_start
print(round(notebook_duration, 2))

10329.29


# ---